In [1]:
import pickle
import textdistance

- 유사도 비교해서 추출한 문장 좌표값 찾기

In [2]:
with open('sdn.pickle', 'rb') as f:
    sdn_list = pickle.load(f)

In [3]:
sdn_list.append('For and on behalf of')

In [4]:
with open('test.pickle', 'rb') as f:
    rect = pickle.load(f)

In [5]:
rect

[{'boundingPoly': {'vertices': [[{'x': 290, 'y': 528},
     {'x': 290, 'y': 699},
     {'x': 329, 'y': 699},
     {'x': 329, 'y': 528}]]},
  'uuid0': 'LETTER'},
 {'boundingPoly': {'vertices': [[{'x': 290, 'y': 716},
     {'x': 290, 'y': 781},
     {'x': 329, 'y': 781},
     {'x': 329, 'y': 716}]]},
  'uuid1': 'OF'},
 {'boundingPoly': {'vertices': [[{'x': 289, 'y': 798},
     {'x': 289, 'y': 1091},
     {'x': 329, 'y': 1091},
     {'x': 329, 'y': 798}]]},
  'uuid2': 'GUARANTEE'},
 {'boundingPoly': {'vertices': [[{'x': 410, 'y': 133},
     {'x': 410, 'y': 219},
     {'x': 435, 'y': 219},
     {'x': 435, 'y': 133}]]},
  'uuid3': 'Shipping'},
 {'boundingPoly': {'vertices': [[{'x': 410, 'y': 228},
     {'x': 410, 'y': 322},
     {'x': 434, 'y': 322},
     {'x': 434, 'y': 228}]]},
  'uuid4': 'Company'},
 {'boundingPoly': {'vertices': [[{'x': 408, 'y': 814},
     {'x': 408, 'y': 850},
     {'x': 430, 'y': 850},
     {'x': 430, 'y': 814}]]},
  'uuid5': 'LG'},
 {'boundingPoly': {'vertices': [[{

In [6]:
# 편집거리 구해주는 함수
def text_similarity_jaro(result):
    import jellyfish
    return [i for i in result for j in sdn_list if jellyfish.jaro_winkler(i,j) > 0.9]

In [7]:
# 문장과, 문장의 좌표를 만들어주는 함수
def make_sentence(rect):
    infertext=[]
    bounding=[]
    for i in range(0, len(rect)):
        infertext.append(rect[i].get(list(rect[i].keys())[-1]))
        bounding.append(rect[i]['boundingPoly']['vertices'][0])
    bounding_mean=[]
    x_bounding_max=[]
    x_bounding_min=[]
    all_x=[] 
    all_y=[]
    for i in range(0, len(bounding)):
        x=[]
        y=[]
        for j in range(0,4):
            y.append(bounding[i][j]['x'])
            x.append(bounding[i][j]['y'])
        bounding_mean.append(sum(y)/len(y))
        x_bounding_max.append(max(x))
        x_bounding_min.append(min(x))
        all_x.append(x)
        all_y.append(y)
    
    words = list(zip(infertext,bounding_mean,x_bounding_max,x_bounding_min))
    
    # y좌표의 평균이 똑같거나 차이가 10 이하인 애들을 한줄로 인식해서 리스트에 담음
    result=[]         # 한 문장을 담는 리스트 
    col=[]
    sentence_bounding={}    # 문장의 bounding를 담는 사전
    y_min=[]
    y_max=[]
    x_min=[]
    x_max=[]
    sentence_verticex=[]   # 문장의 좌표값을 담는 리스트

    i=0
    while i<len(words):
        try:
            # (바로 앞뒤 텍스트의 y좌표 평균이 같거나 10보다 작을경우) 그리고 앞뒤 텍스트의 거리가 20보다 작은 경우
            if abs(words[i][1]-words[i+1][1])<=10 and abs(words[i][2]-words[i+1][3])<=40:
                col.append(words[i][0])
            else:
                col.append(words[i][0])
                result.append(' '.join(col))
                sentence_bounding[' '.join(col)]={'vertices':[]}
                for word in col:
                    y_min.append(min(all_y[infertext.index(word)]))  # 만들어진 문장의 각 단어의 y 최소값을 담음
                    y_max.append(max(all_y[infertext.index(word)]))
                    x_min.append(min(all_x[infertext.index(word)]))
                    x_max.append(max(all_x[infertext.index(word)]))
                    del all_y[infertext.index(word)]
                    del all_x[infertext.index(word)]
                    infertext.remove(word)
                sentence_bounding[' '.join(col)]['vertices'].append({'x':min(x_min), 'y':min(y_min)})
                sentence_bounding[' '.join(col)]['vertices'].append({'x':max(x_max), 'y':min(y_min)})
                sentence_bounding[' '.join(col)]['vertices'].append({'x':max(x_max), 'y':max(y_max)})
                sentence_bounding[' '.join(col)]['vertices'].append({'x':min(x_min), 'y':max(y_max)})
                sentence_verticex.append(sentence_bounding[' '.join(col)]['vertices'])
                y_min=[]
                x_min=[]
                y_max=[]
                x_max=[]
                col=[]
        except: # 맨 뒤에 단어는 비교해줄 단어가 없기 때문에!
            col.append(words[i][0])
            result.append(' '.join(col))
            sentence_bounding[' '.join(col)]={'vertices':[]}
            for word in col:
                y_min.append(min(all_y[infertext.index(word)]))  # 만들어진 문장의 각 단어의 y 최소값을 담음
                y_max.append(max(all_y[infertext.index(word)]))
                x_min.append(min(all_x[infertext.index(word)]))
                x_max.append(max(all_x[infertext.index(word)]))
                del all_y[infertext.index(word)]
                del all_x[infertext.index(word)]
                infertext.remove(word)
            sentence_bounding[' '.join(col)]['vertices'].append({'x':min(x_min), 'y':min(y_min)})
            sentence_bounding[' '.join(col)]['vertices'].append({'x':max(x_max), 'y':min(y_min)})
            sentence_bounding[' '.join(col)]['vertices'].append({'x':max(x_max), 'y':max(y_max)})
            sentence_bounding[' '.join(col)]['vertices'].append({'x':min(x_min), 'y':max(y_max)})
            sentence_verticex.append(sentence_bounding[' '.join(col)]['vertices'])
            y_min=[]
            x_min=[]
            y_max=[]
            x_max=[]
            col=[]
        i+=1
    fixed_result=list(zip(result,sentence_verticex))
    fixed_result
    return result, fixed_result

In [8]:
result, fixed_result = make_sentence(rect)

In [9]:
# 추출된 문장과 문장의 좌표를 리턴해주는 함수
def find_bounding(fixed_result, extraction):
    from collections import defaultdict
    d=defaultdict(list)
    
    for k,v in fixed_result:
        d[k].append(v)
    
    return [d[i] for i in extraction]

- 이거 실행하면 된다

In [10]:
find_bounding(fixed_result,text_similarity_jaro(result))

[[[{'x': 139, 'y': 444},
   {'x': 411, 'y': 444},
   {'x': 411, 'y': 469},
   {'x': 139, 'y': 469}]],
 [[{'x': 363, 'y': 1817},
   {'x': 559, 'y': 1817},
   {'x': 559, 'y': 1850},
   {'x': 363, 'y': 1850}],
  [{'x': 1050, 'y': 1815},
   {'x': 1247, 'y': 1815},
   {'x': 1247, 'y': 1859},
   {'x': 1050, 'y': 1859}]],
 [[{'x': 363, 'y': 1817},
   {'x': 559, 'y': 1817},
   {'x': 559, 'y': 1850},
   {'x': 363, 'y': 1850}],
  [{'x': 1050, 'y': 1815},
   {'x': 1247, 'y': 1815},
   {'x': 1247, 'y': 1859},
   {'x': 1050, 'y': 1859}]]]

# Part2 보완하기

In [11]:
infertext=[]
bounding=[]
for i in range(0, len(rect)):
    infertext.append(rect[i].get(list(rect[i].keys())[-1]))
    bounding.append(rect[i]['boundingPoly']['vertices'][0])
bounding_mean=[]
x_bounding_max=[]
x_bounding_min=[]
all_x=[] 
all_y=[]
for i in range(0, len(bounding)):
    x=[]
    y=[]
    for j in range(0,4):
        y.append(bounding[i][j]['x'])
        x.append(bounding[i][j]['y'])
    bounding_mean.append(sum(y)/len(y))
    x_bounding_max.append(max(x))
    x_bounding_min.append(min(x))
    all_x.append(x)
    all_y.append(y)

words = list(zip(infertext,bounding_mean,x_bounding_max,x_bounding_min))

# y좌표의 평균이 똑같거나 차이가 10 이하인 애들을 한줄로 인식해서 리스트에 담음
result=[]         # 한 문장을 담는 리스트 
col=[]
sentence_bounding={}    # 문장의 bounding를 담는 사전
y_min=[]
y_max=[]
x_min=[]
x_max=[]
sentence_verticex=[]   # 문장의 좌표값 담기

i=0
while i<len(words):
    try:
        # (바로 앞뒤 텍스트의 y좌표 평균이 같거나 10보다 작을경우) 그리고 앞뒤 텍스트의 거리가 20보다 작은 경우
        if abs(words[i][1]-words[i+1][1])<=10 and abs(words[i][2]-words[i+1][3])<=40:
            col.append(words[i][0])
        else:
            col.append(words[i][0])
            result.append(' '.join(col))
            sentence_bounding[' '.join(col)]={'vertices':[]}
            for word in col:
                y_min.append(min(all_y[infertext.index(word)]))  # 만들어진 문장의 각 단어의 y 최소값을 담음
                y_max.append(max(all_y[infertext.index(word)]))
                x_min.append(min(all_x[infertext.index(word)]))
                x_max.append(max(all_x[infertext.index(word)]))
                del all_y[infertext.index(word)]
                del all_x[infertext.index(word)]
                infertext.remove(word)
            sentence_bounding[' '.join(col)]['vertices'].append({'x':min(x_min), 'y':min(y_min)})
            sentence_bounding[' '.join(col)]['vertices'].append({'x':max(x_max), 'y':min(y_min)})
            sentence_bounding[' '.join(col)]['vertices'].append({'x':max(x_max), 'y':max(y_max)})
            sentence_bounding[' '.join(col)]['vertices'].append({'x':min(x_min), 'y':max(y_max)})
            sentence_verticex.append(sentence_bounding[' '.join(col)]['vertices'])
            y_min=[]
            y_max=[]
            x_min=[]
            x_max=[]
            col=[]
    except: # 맨 뒤에 단어는 비교해줄 단어가 없기 때문에!
        col.append(words[i][0])
        result.append(' '.join(col))
        sentence_bounding[' '.join(col)]={'vertices':[]}
        for word in col:
            y_min.append(min(all_y[infertext.index(word)]))  # 만들어진 문장의 각 단어의 y 최소값을 담음
            y_max.append(max(all_y[infertext.index(word)]))
            x_min.append(min(all_x[infertext.index(word)]))
            x_max.append(max(all_x[infertext.index(word)]))
            del all_y[infertext.index(word)]
            del all_x[infertext.index(word)]
            infertext.remove(word)
        sentence_bounding[' '.join(col)]['vertices'].append({'x':min(x_min), 'y':min(y_min)})
        sentence_bounding[' '.join(col)]['vertices'].append({'x':max(x_max), 'y':min(y_min)})
        sentence_bounding[' '.join(col)]['vertices'].append({'x':max(x_max), 'y':max(y_max)})
        sentence_bounding[' '.join(col)]['vertices'].append({'x':min(x_min), 'y':max(y_max)})
        sentence_verticex.append(sentence_bounding[' '.join(col)]['vertices'])
        y_min=[]
        y_max=[]
        x_min=[]
        x_max=[]
        col=[]
    i+=1
    fixed_result=list(zip(result,sentence_verticex))
    fixed_result

In [12]:
result

['LETTER OF GUARANTEE',
 'Shipping Company',
 'LG No',
 'LG Date',
 'ALCALDE LINARES Angel',
 '20200117',
 'Letter of Credit No',
 'Date of Issue',
 'IV NO',
 'Shipper',
 'Consignee',
 'i',
 'Port of Loading',
 'Bill of Lading No',
 'Date of Issue',
 'CHENNAI INDIA',
 '20260103',
 'Port of Discharge or Place of Delivery',
 'Vessel NameVoyage No',
 'BUSAN KOREA',
 'ae',
 'ray oie aes',
 'eneo uso emp',
 'Description of Goods',
 'PHARMACEL 102 S0KG',
 'orn',
 'Whereas you have issued a Bill of Lading covering the above shipmentand the above cargo has been arrived at the above port of',
 'dischargeor the above place of delivery we hereby request you to alve delivery of the said cargo to the above mentioned party',
 'without praduction of the original Bill of Lading',
 'In consideration of your complying with our above request we hereby agree as follows',
 '1 To indemnify you your servants and agents and to noid all of you harness in respect of lability loss damage or',
 'expenses which yo

- 만들어진 문장들 다시 비교

In [13]:
fixed_result

[('LETTER OF GUARANTEE',
  [{'x': 528, 'y': 289},
   {'x': 1091, 'y': 289},
   {'x': 1091, 'y': 329},
   {'x': 528, 'y': 329}]),
 ('Shipping Company',
  [{'x': 133, 'y': 410},
   {'x': 322, 'y': 410},
   {'x': 322, 'y': 435},
   {'x': 133, 'y': 435}]),
 ('LG No',
  [{'x': 814, 'y': 408},
   {'x': 892, 'y': 408},
   {'x': 892, 'y': 430},
   {'x': 814, 'y': 430}]),
 ('LG Date',
  [{'x': 1287, 'y': 406},
   {'x': 1378, 'y': 406},
   {'x': 1378, 'y': 429},
   {'x': 1287, 'y': 429}]),
 ('ALCALDE LINARES Angel',
  [{'x': 139, 'y': 444},
   {'x': 411, 'y': 444},
   {'x': 411, 'y': 469},
   {'x': 139, 'y': 469}]),
 ('20200117',
  [{'x': 1299, 'y': 446},
   {'x': 1460, 'y': 446},
   {'x': 1460, 'y': 467},
   {'x': 1299, 'y': 467}]),
 ('Letter of Credit No',
  [{'x': 814, 'y': 499},
   {'x': 1006, 'y': 499},
   {'x': 1006, 'y': 520},
   {'x': 814, 'y': 520}]),
 ('Date of Issue',
  [{'x': 1288, 'y': 498},
   {'x': 1413, 'y': 498},
   {'x': 1413, 'y': 518},
   {'x': 1288, 'y': 518}]),
 ('IV NO',
 

In [14]:
bounding_mean=[]
x_bounding_max=[]
x_bounding_min=[]
all_x=[] 
all_y=[]

for i in range(0, len(sentence_verticex)):
    x=[]
    y=[]
    for j in range(0,4):
        y.append(bounding[i][j]['y'])
        x.append(bounding[i][j]['x'])
    bounding_mean.append(sum(y)/len(y))
    x_bounding_max.append(max(x))
    x_bounding_min.append(min(x))
    all_x.append(x)
    all_y.append(y)

In [15]:
words = list(zip(result,bounding_mean,x_bounding_max,x_bounding_min))
words

[('LETTER OF GUARANTEE', 613.5, 329, 290),
 ('Shipping Company', 748.5, 329, 290),
 ('LG No', 944.5, 329, 289),
 ('LG Date', 176.0, 435, 410),
 ('ALCALDE LINARES Angel', 275.0, 434, 410),
 ('20200117', 832.0, 430, 408),
 ('Letter of Credit No', 875.0, 428, 409),
 ('Date of Issue', 1305.5, 429, 406),
 ('IV NO', 1354.5, 427, 408),
 ('Shipper', 186.5, 464, 445),
 ('Consignee', 294.0, 468, 445),
 ('i', 382.0, 469, 444),
 ('Port of Loading', 1379.5, 467, 446),
 ('Bill of Lading No', 843.0, 519, 501),
 ('Date of Issue', 889.0, 519, 499),
 ('CHENNAI INDIA', 935.0, 519, 499),
 ('20260103', 989.0, 520, 501),
 ('Port of Discharge or Place of Delivery', 1311.5, 518, 499),
 ('Vessel NameVoyage No', 1350.5, 518, 498),
 ('BUSAN KOREA', 1390.0, 518, 499),
 ('ae', 842.5, 556, 535),
 ('ray oie aes', 897.5, 556, 535),
 ('eneo uso emp', 170.0, 619, 588),
 ('Description of Goods', 865.5, 612, 590),
 ('PHARMACEL 102 S0KG', 910.5, 721, 707),
 ('orn', 155.0, 724, 705),
 ('Whereas you have issued a Bill of La

In [16]:
# y좌표의 평균이 똑같거나 차이가 10 이하인 애들을 한줄로 인식해서 리스트에 담음
sentence=[]         # 한 문장을 담는 리스트 
col=[]
sentence_bounding={}    # 문장의 bounding를 담는 사전
y_min=[]
y_max=[]
x_min=[]
x_max=[]
sentence_verticex=[]   # 문장의 좌표값 담기

i=0
# Words (문장, Y평균, X 최대, X 최소)
while i<len(words):
    try:
        # (바로 앞뒤 텍스트의 y좌표 평균이 같거나 10보다 작을경우) 그리고 앞뒤 텍스트의 거리가 20보다 작은 경우
        if abs(words[i][1]-words[i+1][1])<=50 and abs(words[i][2]-words[i+1][3])<=20:
            col.append(words[i][0])
        else:
            col.append(words[i][0])
            sentence.append(' '.join(col))
            sentence_bounding[' '.join(col)]={'vertices':[]}
            for word in col:
                y_min.append(min(all_y[result.index(word)]))  # 만들어진 문장의 각 단어의 y 최소값을 담음
                y_max.append(max(all_y[result.index(word)]))
                x_min.append(min(all_x[result.index(word)]))
                x_max.append(max(all_x[result.index(word)]))
                del all_y[result.index(word)]
                del all_x[result.index(word)]
                result.remove(word)
            sentence_bounding[' '.join(col)]['vertices'].append({'x':min(x_min), 'y':min(y_min)})
            sentence_bounding[' '.join(col)]['vertices'].append({'x':max(x_max), 'y':min(y_min)})
            sentence_bounding[' '.join(col)]['vertices'].append({'x':max(x_max), 'y':max(y_max)})
            sentence_bounding[' '.join(col)]['vertices'].append({'x':min(x_min), 'y':max(y_max)})
            sentence_verticex.append(sentence_bounding[' '.join(col)]['vertices'])
            y_min=[]
            y_max=[]
            x_min=[]
            x_max=[]
            col=[]
    except: # 맨 뒤에 단어는 비교해줄 단어가 없기 때문에!
        col.append(words[i][0])
        sentence.append(' '.join(col))
        sentence_bounding[' '.join(col)]={'vertices':[]}
        for word in col:
            y_min.append(min(all_y[result.index(word)]))  # 만들어진 문장의 각 단어의 y 최소값을 담음
            y_max.append(max(all_y[result.index(word)]))
            x_min.append(min(all_x[result.index(word)]))
            x_max.append(max(all_x[result.index(word)]))
            del all_y[result.index(word)]
            del all_x[result.index(word)]
            result.remove(word)
        sentence_bounding[' '.join(col)]['vertices'].append({'x':min(x_min), 'y':min(y_min)})
        sentence_bounding[' '.join(col)]['vertices'].append({'x':max(x_max), 'y':min(y_min)})
        sentence_bounding[' '.join(col)]['vertices'].append({'x':max(x_max), 'y':max(y_max)})
        sentence_bounding[' '.join(col)]['vertices'].append({'x':min(x_min), 'y':max(y_max)})
        sentence_verticex.append(sentence_bounding[' '.join(col)]['vertices'])
        y_min=[]
        y_max=[]
        x_min=[]
        x_max=[]
        col=[]
    i+=1

In [17]:
sentence

['LETTER OF GUARANTEE',
 'Shipping Company',
 'LG No',
 'LG Date',
 'ALCALDE LINARES Angel',
 '20200117',
 'Letter of Credit No',
 'Date of Issue',
 'IV NO',
 'Shipper',
 'Consignee',
 'i',
 'Port of Loading',
 'Bill of Lading No Date of Issue CHENNAI INDIA',
 '20260103',
 'Port of Discharge or Place of Delivery Vessel NameVoyage No BUSAN KOREA',
 'ae',
 'ray oie aes',
 'eneo uso emp',
 'Description of Goods',
 'PHARMACEL 102 S0KG',
 'orn',
 'Whereas you have issued a Bill of Lading covering the above shipmentand the above cargo has been arrived at the above port of',
 'dischargeor the above place of delivery we hereby request you to alve delivery of the said cargo to the above mentioned party',
 'without praduction of the original Bill of Lading In consideration of your complying with our above request we hereby agree as follows 1 To indemnify you your servants and agents and to noid all of you harness in respect of lability loss damage or',
 'expenses which you may sustain by reason 

- Y 중간값으로 가보자

In [18]:
import statistics
infertext=[]
bounding=[]
for i in range(0, len(rect)):
    infertext.append(rect[i].get(list(rect[i].keys())[-1]))
    bounding.append(rect[i]['boundingPoly']['vertices'][0])
bounding_median=[]
x_bounding_max=[]
x_bounding_min=[]
all_x=[] 
all_y=[]
for i in range(0, len(bounding)):
    x=[]
    y=[]
    for j in range(0,4):
        y.append(bounding[i][j]['x'])
        x.append(bounding[i][j]['y'])
    bounding_median.append(statistics.median(y))
    x_bounding_max.append(max(x))
    x_bounding_min.append(min(x))
    all_x.append(x)
    all_y.append(y)

words = list(zip(infertext,bounding_median,x_bounding_max,x_bounding_min))

# y좌표의 평균이 똑같거나 차이가 10 이하인 애들을 한줄로 인식해서 리스트에 담음
result=[]         # 한 문장을 담는 리스트 
col=[]
sentence_bounding={}    # 문장의 bounding를 담는 사전
y_min=[]
y_max=[]
x_min=[]
x_max=[]
sentence_verticex=[]   # 문장의 좌표값 담기

i=0
while i<len(words):
    try:
        # (바로 앞뒤 텍스트의 y좌표 평균이 같거나 10보다 작을경우) 그리고 앞뒤 텍스트의 거리가 20보다 작은 경우
        if abs(words[i][1]-words[i+1][1])<=15 and abs(words[i][2]-words[i+1][3])<=40:
            col.append(words[i][0])
        else:
            col.append(words[i][0])
            result.append(' '.join(col))
            sentence_bounding[' '.join(col)]={'vertices':[]}
            for word in col:
                y_min.append(min(all_y[infertext.index(word)]))  # 만들어진 문장의 각 단어의 y 최소값을 담음
                y_max.append(max(all_y[infertext.index(word)]))
                x_min.append(min(all_x[infertext.index(word)]))
                x_max.append(max(all_x[infertext.index(word)]))
                del all_y[infertext.index(word)]
                del all_x[infertext.index(word)]
                infertext.remove(word)
            sentence_bounding[' '.join(col)]['vertices'].append({'x':min(x_min), 'y':min(y_min)})
            sentence_bounding[' '.join(col)]['vertices'].append({'x':max(x_max), 'y':min(y_min)})
            sentence_bounding[' '.join(col)]['vertices'].append({'x':max(x_max), 'y':max(y_max)})
            sentence_bounding[' '.join(col)]['vertices'].append({'x':min(x_min), 'y':max(y_max)})
            sentence_verticex.append(sentence_bounding[' '.join(col)]['vertices'])
            y_min=[]
            y_max=[]
            x_min=[]
            x_max=[]
            col=[]
    except: # 맨 뒤에 단어는 비교해줄 단어가 없기 때문에!
        col.append(words[i][0])
        result.append(' '.join(col))
        sentence_bounding[' '.join(col)]={'vertices':[]}
        for word in col:
            y_min.append(min(all_y[infertext.index(word)]))  # 만들어진 문장의 각 단어의 y 최소값을 담음
            y_max.append(max(all_y[infertext.index(word)]))
            x_min.append(min(all_x[infertext.index(word)]))
            x_max.append(max(all_x[infertext.index(word)]))
            del all_y[infertext.index(word)]
            del all_x[infertext.index(word)]
            infertext.remove(word)
        sentence_bounding[' '.join(col)]['vertices'].append({'x':min(x_min), 'y':min(y_min)})
        sentence_bounding[' '.join(col)]['vertices'].append({'x':max(x_max), 'y':min(y_min)})
        sentence_bounding[' '.join(col)]['vertices'].append({'x':max(x_max), 'y':max(y_max)})
        sentence_bounding[' '.join(col)]['vertices'].append({'x':min(x_min), 'y':max(y_max)})
        sentence_verticex.append(sentence_bounding[' '.join(col)]['vertices'])
        y_min=[]
        y_max=[]
        x_min=[]
        x_max=[]
        col=[]
    i+=1
    fixed_result=list(zip(result,sentence_verticex))
    fixed_result

In [19]:
words

[('LETTER', 309.5, 699, 528),
 ('OF', 309.5, 781, 716),
 ('GUARANTEE', 309.0, 1091, 798),
 ('Shipping', 422.5, 219, 133),
 ('Company', 422.0, 322, 228),
 ('LG', 419.0, 850, 814),
 ('No', 418.5, 892, 858),
 ('LG', 417.5, 1324, 1287),
 ('Date', 417.5, 1378, 1331),
 ('ALCALDE', 454.5, 234, 139),
 ('LINARES', 456.5, 343, 245),
 ('Angel', 456.5, 411, 353),
 ('20200117', 456.5, 1460, 1299),
 ('Letter', 510.0, 872, 814),
 ('of', 509.0, 899, 879),
 ('Credit', 509.0, 964, 906),
 ('No', 510.5, 1006, 972),
 ('Date', 508.5, 1335, 1288),
 ('of', 508.0, 1361, 1340),
 ('Issue', 508.5, 1413, 1367),
 ('IV', 545.5, 858, 827),
 ('NO', 545.5, 918, 877),
 ('Shipper', 603.5, 210, 130),
 ('Consignee', 601.0, 916, 815),
 ('i', 714.0, 912, 909),
 ('Port', 714.5, 175, 135),
 ('of', 713.0, 203, 181),
 ('Loading', 716.0, 287, 209),
 ('Bill', 711.5, 840, 815),
 ('of', 712.0, 869, 847),
 ('Lading', 714.0, 938, 875),
 ('No', 712.0, 974, 945),
 ('Date', 711.5, 1334, 1287),
 ('of', 710.5, 1361, 1340),
 ('Issue', 711.0

In [20]:
result

['LETTER OF GUARANTEE',
 'Shipping Company',
 'LG No',
 'LG Date',
 'ALCALDE LINARES Angel',
 '20200117',
 'Letter of Credit No',
 'Date of Issue',
 'IV NO',
 'Shipper',
 'Consignee',
 'i',
 'Port of Loading',
 'Bill of Lading No',
 'Date of Issue',
 'CHENNAI INDIA',
 '20260103',
 'Port of Discharge or Place of Delivery',
 'Vessel NameVoyage No',
 'BUSAN KOREA',
 'ae',
 'ray oie aes',
 'eneo uso emp',
 'Description of Goods',
 'PHARMACEL 102 S0KG',
 'orn',
 'Whereas you have issued a Bill of Lading covering the above shipmentand the above cargo has been arrived at the above port of',
 'dischargeor the above place of delivery we hereby request you to alve delivery of the said cargo to the above mentioned party',
 'without praduction of the original Bill of Lading',
 'In consideration of your complying with our above request we hereby agree as follows',
 '1 To indemnify you your servants and agents and to noid all of you harness in respect of lability loss damage or',
 'expenses which yo

- 시행착오를 거쳐서 여러 줄도 한 문장으로 시도해 볼 수 있을 것 같다